In [20]:
import json
import web3

from web3 import Web3
from solc import compile_source
from web3.contract import ConciseContract

In [21]:
# Solidity source code
contract_source_code = '''
pragma solidity >0.4.0;

contract simplestorage {

  int public familyBalance;
  address public otherParentAddr;
  address public balanceSetterAddr;

  constructor(address parentAddr) public {
    familyBalance = 0;
    balanceSetterAddr = msg.sender;
    otherParentAddr = parentAddr;
  }

  function makeRecord(int x) public {
    if (x <= 0) {
        familyBalance = familyBalance + x;
    }
    else {
        if (msg.sender == balanceSetterAddr || msg.sender == otherParentAddr) {
            familyBalance = familyBalance + x;
        }
    }
  }

  function getBalance() public view returns (int retVal) {
    return familyBalance;
  }

}
'''
# Compiled source code
compiled_sol = compile_source(contract_source_code) 
contract_interface = compiled_sol['<stdin>:simplestorage']

In [22]:
contract_interface['bin']

'608060405234801561001057600080fd5b506040516020806103ce833981018060405281019080805190602001909291905050506000808190555033600260006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555080600160006101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff16021790555050610302806100cc6000396000f30060806040526004361061006d576000357c0100000000000000000000000000000000000000000000000000000000900463ffffffff16806312065fe0146100725780638b1579d71461009d5780639c7ed1af146100f4578063ebe4b0031461014b578063ecf7ee4a14610176575b600080fd5b34801561007e57600080fd5b506100876101a3565b6040518082815260200191505060405180910390f35b3480156100a957600080fd5b506100b26101ac565b604051808273ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff16815260200191505060405180910390f35b34801561010057600080fd5b506101096101d2565b604051808273ffffffffffffffffffffffffffffffffffffffff1673fff

In [23]:
NODE_ADDR = 'http://XXX.XXX.p2pify.com:8545'

In [24]:
# web3.py instance
w3 = Web3(Web3.HTTPProvider(NODE_ADDR))

In [25]:
# inject the poa compatibility middleware to the innermost layer
from web3.middleware import geth_poa_middleware
w3.middleware_stack.inject(geth_poa_middleware, layer=0)

In [26]:
w3.version.node

'Geth/v1.8.12-stable-c6f0ae4e/linux-amd64/go1.9.7'

In [27]:
# set pre-funded account as sender
w3.eth.defaultAccount = w3.eth.accounts[0]
'Account:', w3.eth.defaultAccount

('Account:', '0x76275BfA3E040e695C03aCBA7BBf34507AAe8E84')

In [28]:
# Instantiate and deploy contract
simplestorage = w3.eth.contract(
    abi=contract_interface['abi'],
    bytecode=contract_interface['bin']
)

In [29]:
# Mom's addr here
otherParentAddr = '0x9664b1d0207b6eaea44d7141a322f659cf580816' 

In [30]:
tx_hash = simplestorage.constructor(Web3.toChecksumAddress(otherParentAddr)).transact()

In [31]:
tx_hash

HexBytes('0x7ba94d3666699a32126c954fda9cb4c86e22e1d184432508728959cb7a174524')

In [32]:
# Wait for the transaction to be mined, and get the transaction receipt
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)

In [33]:
'Block number:', tx_receipt.blockNumber

('Block number:', 3985)

In [34]:
'Contract address:', tx_receipt.contractAddress

('Contract address:', '0xa2B5e8e28e7E0187ee955b04C48f715ac43fcE08')

In [35]:
simplestorage = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=contract_interface['abi'],
)

In [37]:
'Balance:', simplestorage.functions.getBalance().call()

('Balance:', 1000)